<a href="https://colab.research.google.com/github/Ardaolmez/ArdaGpt/blob/main/arda_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this section, I cleaned the data from emojis and I created list for cleaned data. I splited the into lines.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

with open('/content/WhatsApp Chat with Ana.txt', 'r', encoding='utf-8') as f0:
    text0 = f0.read()
with open('/content/WhatsApp Chat with Dogu.txt', 'r', encoding='utf-8') as f1:
    text1= f1.read()
with open('/content/WhatsApp Chat with Hatun.txt', 'r', encoding='utf-8') as f2:
    text2= f2.read()
text= text0+text1+text2

moj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
moj.sub('',text)
textlist=text.splitlines()
print(textlist[-100:])


['01/06/2023, 15:48 - Hatun: Geldik', '01/06/2023, 15:49 - Arda: Sedef ders caliscak m', '01/06/2023, 15:49 - Arda: Nu', '01/06/2023, 15:49 - Arda: Mi', '01/06/2023, 15:49 - Hatun: Evet', '01/06/2023, 15:49 - Arda: Iki', '01/06/2023, 15:49 - Hatun: İki mi', '01/06/2023, 15:50 - Arda: Okiii', '01/06/2023, 18:18 - Hatun: Ben geldim masaya', '01/06/2023, 21:28 - Hatun: Arda', '01/06/2023, 21:29 - Arda: Aslkiii', '01/06/2023, 21:30 - Hatun: Kutluhan beni sinir etti', '01/06/2023, 21:35 - Arda: Noldu ya', '01/06/2023, 21:35 - Hatun: Fayansları silmiş', '01/06/2023, 21:36 - Hatun: Ne zaman geleceksiniz bulaşıkları yıkayın', '01/06/2023, 21:36 - Hatun: Bir dolabı silip temizleyelim diyo', '01/06/2023, 21:36 - Hatun: O kadar sinirlendim ki kendisi 2 ay yatıyor hiçbir bok yapmıyor', '01/06/2023, 21:37 - Arda: Asliiiiii bozma ya hiç gelince silerim ben muhatap olma bjle', '01/06/2023, 21:53 - Hatun: Zaten ben berradayım ama şimdi berraya geldim', '01/06/2023, 21:54 - Arda: Ben donuom sikdi', '01

In this section, I created a new list to add my and my mothers's messages. Because, I had a small size text messages, I added my mom's text also. Characteristics of our messages are similar.

In [ ]:
length=len(textlist)
print(length)
print(textlist[length-2])
textlist0=[]
for i in range(length):
  if "Arda:" in textlist[i]:
    textlist0.append(textlist[i])
  if 'Ana' in textlist[i]:
    textlist0.append(textlist[i])

86513
02/06/2023, 16:16 - Hatun: Geliyorum


In this section, I cleaned the data from date information and medias.

In [ ]:
for i,line in enumerate(textlist0):
  if 'Arda:' in line:
    textlist0[i]=line[26:]
  else:
     textlist0[i]=line[25:]

for line in textlist0:
  if "<Media omitted>" in line:
    textlist0.remove(line)



In this section, I find all characters which were used in text messages.

In [ ]:
text='\n'.join(textlist0)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ\]^_abcdefghijklmnopqrstuvwxyz{ £²ÇÖ×ÜßàäçîñöúüýďğİıĺőŞşž’⁶€☺♥✌❤️🌹🍀🍌🍎🍏👀💃💐💝💫😂😅😈😊😎😘😜😡😢😭🙃🙏🤓🤣🤨🤩🤭🥰🥲🥳🥴🥹🦍🧐🧿
161


stoi: Map strings to integers. Dictionary for every character assign a number

ito: Map integers to strings. Dictionary for every integer get the character

(Normally, we use pretrained tokenizer from libraries like huggingface.)


In [ ]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode(text[:100]))


[37, 65, 78, 2, 65, 82, 84, 73, 75, 2, 87, 72, 65, 84, 83, 85, 80, 2, 43, 2, 66, 85, 82, 65, 68, 65, 78, 2, 75, 85, 76, 76, 65, 78, 112, 67, 65, 77, 1, 36, 69, 78, 2, 65, 78, 78, 69, 78, 1, 54, 65, 77, 65, 77, 68, 85, 82, 2, 65, 78, 78, 69, 1, 35, 78, 78, 78, 69, 1, 48, 65, 80, 73, 89, 79, 82, 83, 85, 78, 1, 37, 65, 78, 73, 77, 1, 35, 82, 68, 65, 67, 73, 77, 1, 47, 73, 83, 83, 69, 68]


In this section, we encode our list from my and my mother's messages.

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([893878]) torch.int64
tensor([37, 65, 78,  2, 65, 82, 84, 73, 75,  2])


In this section, we split the data into training and validation lists.

In [ ]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In this section, we define our hyperparameters and we have a get_batch()
to create our batches. Bathces are samples from our data. We use batches to have a faster training, otherwise we use all data in every step. estimate_loss(): calculate losses in every step. Loss estimation no need to take gradients, that is why we use torch.no_grad()


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 8
block_size = 32
max_iters = 10000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

This is our artitechture. Imagine you're shopping for a new pair of shoes on an e-commerce website. The website's recommendation system uses the key-value-query concept. When you enter your search query (e.g., "running shoes"), the system maps this query against a set of keys associated with various shoe products in their database. These keys could include product names, descriptions, brand names, and more.

The attention operation, in this context, can be thought of as a retrieval process too.

The attention mechanism calculates a weighted average of the values, where each value corresponds to a different shoe product. It does so by assigning weights to each product based on how well they match your query. So, the resulting recommendation you see is akin to retrieving the most relevant shoes from the database based on the weighted values assigned during the attention process. If you remove the constraint of one-hot weighting (where only one product is selected), it becomes a proportional retrieval, with products being retrieved based on their probability weights in the attention calculation.

In [ ]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # masking future tokens
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out

Here, we call head, multiple times. In this way, we have deeper model

In [ ]:

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


The model name is bigram model. It is not related with bigram models directly. Biagram models are the simplest approach for this problem

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        #we use two embeddings here.
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In this, section we have training loop. At the end, we print the results.

In [ ]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        #print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.222113 M parameters
	asızmalim at bakalya
Babım bi dovustuyorum şarhı ya marahasift vatla ateri. Sen de sikge indisi yatmiyim?
htemlikey
Tub bak
Majil
Gorcur
Yormaniniyor
Yogde
Amster benim hazimbin oremaldim ne
Bölüm seyfz lan
Eviyyorum
Jsjjejsjjjskjejsjm
Buuuuuuum
yardi adam gitti
Bol tani
Sjvta
Tent tanin merakla Baka olsun tamardığinu daha artik aranamist yok oylek
1 dencriidm
Bisi adam
Diletifylim bu anne
Gitmesan gitiorum
Ama bir ka nedim
Gullsen  balac
Sarar fana çok yanindamm
Datioscagijiz i
Kcrosin
Yok belali youydu
Ben balqvarun olaramiz
Ne bittius
Evettliymesim
5öyje di bugun varayin
Kerisi
Okeydim
Res dece golden vet donuon gelmiyorum
J
Hayio ne en tr i degkli mi
Bunacadikgaldi deyide
Kart var
Knka bu anne
<Media omitted>
Da deleter
Kelt ka
Bibi ahtasap da yapaliodun oni
Aem gozdeksin lan
50 tak hayatta 
Rarriliil
Kolayim duytum gundekin
Hayir anlatıyim
Jks bin
Prik anfall
Rersuller olarima mapli olurdum
Ne knka
Orda hall
Hahah
Xmkl
Buuuuum
Ben yaraki fanemsi
Gitti
25
Gen